# SBAC Dashboard Predictor

The purpose of this Jupyter notebook is to take the raw data file from SBAC and a query from Aeries as well as some inputs, the dashboard prediction for ELA and Math will be calculated. A raw data file will also be generated that gives the Distance From Standard (DFS) for each student.

In [ ]:
import numpy as np
import pandas as pd

## Loading Data

__Aeries Query__: LIST STU SC ID CID NM GR LF SPECIALED DISADVANTAGED STU.ETH? STU.RC1? LAC LAC.RD1

After running the query through Aeries to get roster and subgroup information, you need to copy the file pathway below as well as the one for the raw SBAC file. The two files will be merged with an inner merge to just leave the students remaining that are enrolled.

In [ ]:
# LIST STU SC ID CID NM GR LF SPECIALED DISADVANTAGED STU.ETH? STU.RC1? LAC LAC.RD1

roster = pd.read_excel(r"C:\Users\derek.castleman\Desktop\PrintQueryToExcel_20240711_134159_0135440.xlsx") # Aeries Query file

sbac = pd.read_csv(r"C:\Users\derek.castleman\Desktop\SBAC_2021-2022.csv") # Raw SBAC file

In [ ]:
roster

In [ ]:
sbac

In [ ]:
merge = pd.merge(roster, sbac, how='inner', left_on='State Student ID', right_on='StudentIdentifier')
merge

In [ ]:
merge = merge.drop_duplicates(subset=['State Student ID', 'Subject'])
merge

In [ ]:
merge.info()

## Redesignated English Learners

Redesignated English Learners within their first four years are counted toward EL subgroup for the dashboard. In order to properly calculate this group, the number of years between redisgnated and test date will be looked at. Students with less than 4 years between designation and tests will be given a new category RL to allow for them to be found.

In [ ]:
# Getting rid of the time and just leaving the date for when the test was submitted
merge['SubmitDateTime'] = merge['SubmitDateTime'].apply(lambda x: x.split(' ', 1)[0])
merge

In [ ]:
# Change submission to datetime
merge['SubmitDateTime']= pd.to_datetime(merge['SubmitDateTime']) 
merge

In [ ]:
# Selecting only redesignated students
reclassified = merge.dropna(subset=['Redes Date'])
reclassified

In [ ]:
reclassified.info()

In [ ]:
# Turn redesignated date to a datetime
reclassified['Redes Date']= pd.to_datetime(reclassified['Redes Date']) 
reclassified

In [ ]:
# Find number of years between redesignation and test
reclassified['Years'] = reclassified['SubmitDateTime'].dt.year - reclassified['Redes Date'].dt.year

In [ ]:
reclassified

In [ ]:
# Selecting students under four years of redesignation
newly_reclassified = reclassified[reclassified['Years'] < 4]
newly_reclassified

In [ ]:
# Selecting students with more than four years after redesignation
old_reclassified = reclassified[reclassified['Years'] >= 4]
old_reclassified

In [ ]:
# Categorize newly redesignated as RL
newly_reclassified['LangFlu'] = 'RL'
newly_reclassified

In [ ]:
# Concats the two redesignated dataframes back together
reclassified = pd.concat([newly_reclassified, old_reclassified])
reclassified

In [ ]:
# Drop the years column
reclassified = reclassified.drop('Years', axis=1)
reclassified

In [ ]:
# Find the non-reclassified students
non_reclassified = merge[merge['Redes Date'].isna()]
non_reclassified

In [ ]:
# Combined non-reclassified with the fixed reclassified dataframe
merge = pd.concat([non_reclassified, reclassified])
merge

## Filtering Data

Through a couples of inputs the person using the code can select which school sites they are interested in and if they want to look at a certain subgroup

In [ ]:
x = input('What are you interest in (All, Elementary, Secondary, Middle, High?             )') # Input choice of site

In [ ]:
if x == 'All': #Selects all the schools at a site
    merge = merge[(merge['School'] == 1) | (merge['School'] == 2) | (merge['School'] == 4) | 
                 (merge['School'] == 6) | (merge['School'] == 7) | (merge['School'] == 8)]
elif x == 'Elementary': #Selects just elementary schools
    merge = merge[(merge['School'] == 4) | (merge['School'] == 6)]
elif x == 'Secondary': # Selects the middle and high school
    merge = merge[(merge['School'] == 1) | (merge['School'] == 2) | (merge['School'] == 7) | (merge['School'] == 8)]
elif x == 'Middle': # Selects just the middle school
    merge = merge[(merge['School'] == 7) | (merge['School'] == 2)]
elif x == 'High': # Selects just the high school
    merge = merge[(merge['School'] == 1) | (merge['School'] == 6)]

In [ ]:
merge

In [ ]:
y = input('All or Subgroups (All, EL, SPED, SED, Hispanic, White, Filipino?             )') # Input subgroup

In [ ]:
if y == 'All':
    merge = merge
elif y == 'EL': # Selects EL and newly redesignated students
    merge = merge[(merge['LangFlu'] == 'L') | (merge['LangFlu'] == 'RL') ]
elif y == 'SPED': # Selects SPED students
    merge = merge[merge['SPECIALED Value'] == 'Yes']
elif y == 'SED': # Selects socioeconomic disadvantaged
    merge = merge[merge['DISADVANTAGED Value'] == 'Yes']
elif y == 'Hispanic': # Selectes Hispanic students
    merge = merge[merge['HispanicOrLatinoEthnicity'] == 'Yes']
elif y == 'White': # Selects white students
    merge = merge[merge['White'] == 'Yes']
elif y == 'Filipino': # Selects Filipino students
    merge = merge[merge['Filipino'] == 'Yes']

In [ ]:
merge

## ELA

The SBAC test will first be selected for ELA and the columns cut down to just the student information and performance. A function is created to create a column that will calculate the DFS for a student based on their grade and performance of SBAC. A sum of the DFS will be generated.

An input will allow for the amount of loss that wants to be included in the calculation.

The ELA predictor for the dashboard will be calculated by adding the sum to the loss and then dividing by the total number of students added to the count of loss.

In [ ]:
ela = merge[merge['Subject'] == 'ELA'] # Select ELA
ela

In [ ]:
ela = ela[ela['AssessmentType'] == 'Summative'] # Select the summative SBAC
ela

In [ ]:
ela = ela[['School', 'Student ID', 'Student Name', 'GradeLevelWhenAssessed', 'Subject', 'LangFlu', 'SPECIALED Value', 
          'DISADVANTAGED Value', 'Description_STU_ETH', 'Description_STU_RC1', 'ScaleScoreAchievementLevel', 
          'ScaleScore']] # Cut it down to columns of interest
ela

In [ ]:
ela.info()

In [ ]:
# Create a function that selects the grade levels for site and generates a DFS column based on grade level
if x == 'All':
    a = ela[ela['GradeLevelWhenAssessed'] == '3']
    a['DFS'] = a['ScaleScore'] - 2432
    b = ela[ela['GradeLevelWhenAssessed'] =='4']
    b['DFS'] = b['ScaleScore'] - 2473
    c = ela[ela['GradeLevelWhenAssessed'] =='5']
    c['DFS'] = c['ScaleScore'] - 2502
    d = ela[ela['GradeLevelWhenAssessed'] == '6']
    d['DFS'] = d['ScaleScore'] - 2531
    e = ela[ela['GradeLevelWhenAssessed'] == '7']
    e['DFS'] = e['ScaleScore'] - 2552
    f = ela[ela['GradeLevelWhenAssessed'] == '8']
    f['DFS'] = f['ScaleScore'] - 2567
    g = ela[ela['GradeLevelWhenAssessed'] == '11']
    g['DFS'] = g['ScaleScore'] - 2583
    ela = pd.concat([a, b, c, d, e, f, g])
elif x == 'Elementary':
    a = ela[ela['GradeLevelWhenAssessed'] == 3]
    a['DFS'] = a['ScaleScore'] - 2432
    b = ela[ela['GradeLevelWhenAssessed'] == 4]
    b['DFS'] = b['ScaleScore'] - 2473
    c = ela[ela['GradeLevelWhenAssessed'] == 5]
    c['DFS'] = c['ScaleScore'] - 2502
    d = ela[ela['GradeLevelWhenAssessed'] == 6]
    d['DFS'] = d['ScaleScore'] - 2531
    ela = pd.concat([a, b, c, d])
elif x == 'Secondary':
    e = ela[ela['GradeLevelWhenAssessed'] == 7]
    e['DFS'] = e['ScaleScore'] - 2552
    f = ela[ela['GradeLevelWhenAssessed'] == 8]
    f['DFS'] = f['ScaleScore'] - 2567
    g = ela[ela['GradeLevelWhenAssessed'] == 11]
    g['DFS'] = g['ScaleScore'] - 2583
    ela = pd.concat([e, f, g])
elif x == 'Middle':
    d = ela[ela['GradeLevelWhenAssessed'] == 6]
    d['DFS'] = d['ScaleScore'] - 2531
    e = ela[ela['GradeLevelWhenAssessed'] == 7]
    e['DFS'] = e['ScaleScore'] - 2552
    f = ela[ela['GradeLevelWhenAssessed'] == 8]
    f['DFS'] = f['ScaleScore'] - 2567
    ela = pd.concat([d, e, f])
elif x == 'High':
    g = ela[ela['GradeLevelWhenAssessed'] == 11]
    g['DFS'] = g['ScaleScore'] - 2583
    ela = g

In [ ]:
ela

In [ ]:
ela_dashboard = ela.groupby(['GradeLevelWhenAssessed'])['DFS'].mean().reset_index
ela_dashboard

In [ ]:
ela_number = ela.groupby(['GradeLevelWhenAssessed'])['DFS'].count().reset_index
ela_number

## Math

The same steps that were followed for the ELA section will now be done for the Math portion of the test

In [ ]:
math = merge[merge['Subject'] == 'Math'] # Selects for Math
math

In [ ]:
math = math[math['AssessmentType'] == 'Summative'] # Selects the summative SBAC
math

In [ ]:
math = math[['School', 'Student ID', 'Student Name', 'GradeLevelWhenAssessed', 'Subject', 'LangFlu', 'SPECIALED Value', 
          'DISADVANTAGED Value', 'Description_STU_ETH', 'Description_STU_RC1', 'ScaleScoreAchievementLevel', 
          'ScaleScore']] # Selects the columns of interest
math

In [ ]:
math.info()

In [ ]:
# Create a function the generates a column for the DFS based on the grade level of a student.
if x == 'All':
    a = math[math['GradeLevelWhenAssessed'] == '03']
    a['DFS'] = a['ScaleScore'] - 2436
    b = math[math['GradeLevelWhenAssessed'] == '04']
    b['DFS'] = b['ScaleScore'] - 2485
    c = math[math['GradeLevelWhenAssessed'] == '05']
    c['DFS'] = c['ScaleScore'] - 2528
    d = math[math['GradeLevelWhenAssessed'] == '06']
    d['DFS'] = d['ScaleScore'] - 2552
    e = math[math['GradeLevelWhenAssessed'] == '07']
    e['DFS'] = e['ScaleScore'] - 2567
    f = math[math['GradeLevelWhenAssessed'] == '08']
    f['DFS'] = f['ScaleScore'] - 2586
    g = math[math['GradeLevelWhenAssessed'] == '11']
    g['DFS'] = g['ScaleScore'] - 2628
    math = pd.concat([a, b, c, d, e, f, g])
elif x == 'Elementary':
    a = math[math['GradeLevelWhenAssessed'] == 3]
    a['DFS'] = a['ScaleScore'] - 2436
    b = math[math['GradeLevelWhenAssessed'] == 4]
    b['DFS'] = b['ScaleScore'] - 2485
    c = math[math['GradeLevelWhenAssessed'] == 5]
    c['DFS'] = c['ScaleScore'] - 2528
    d = math[math['GradeLevelWhenAssessed'] == 6]
    d['DFS'] = d['ScaleScore'] - 2552
    math = pd.concat([a, b, c, d])
elif x == 'Secondary':
    e = math[math['GradeLevelWhenAssessed'] == 7]
    e['DFS'] = e['ScaleScore'] - 2567
    f = math[math['GradeLevelWhenAssessed'] == 8]
    f['DFS'] = f['ScaleScore'] - 2586
    g = math[math['GradeLevelWhenAssessed'] == 11]
    g['DFS'] = g['ScaleScore'] - 2628
    math = pd.concat([e, f, g])
elif x == 'Middle':
    d = math[math['GradeLevelWhenAssessed'] == 6]
    d['DFS'] = d['ScaleScore'] - 2552
    e = math[math['GradeLevelWhenAssessed'] == 7]
    e['DFS'] = e['ScaleScore'] - 2567
    f = math[math['GradeLevelWhenAssessed'] == 8]
    f['DFS'] = f['ScaleScore'] - 2586
    math = pd.concat([d, e, f])
elif x == 'High':
    g = math[math['GradeLevelWhenAssessed'] == 11]
    g['DFS'] = g['ScaleScore'] - 2628
    math = g

In [ ]:
math

In [ ]:
math_dashboard = math.groupby(['GradeLevelWhenAssessed'])['DFS'].mean().reset_index
math_dashboard

In [ ]:
math_number = math.groupby(['GradeLevelWhenAssessed'])['DFS'].count().reset_index
math_number

In [ ]:
# Create one final datatable with ELA and Math and DFS for students
final = pd.concat([ela, math])
final

In [ ]:
final['LangFlu'] = final['LangFlu'].replace('RL', 'R') # Replaces RL students with original R
final

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "SBAC Student DFS", filename = "SBAC Student DFS"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final)

In [ ]:
# ELA dashboard prediction
ela_dashboard

In [ ]:
ela_number

In [ ]:
# Math dashboard prediction
math_dashboard

In [ ]:
math_number